# Orizon: Frequência de Exames Adicionais

In [2]:
import findspark
findspark.init("/usr/local/spark")
from pyspark.sql import SparkSession, SQLContext
from collections import Counter
from pandas import DataFrame

Configurando sessão:

In [3]:
spark = SparkSession.builder    \
        .master("local")    \
        .appName("Linear Regression Model")    \
        .config("spark.executor.memory", "1gb")    \
        .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

Carregando dados do TUSS relativo à displasia coxo-femural (código 30724058)

In [4]:
csv_path = "../orizon/extract/data_30724058.csv"
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(csv_path)

Verificando se os tipos foram inferidos corretamente:

In [5]:
df.printSchema()

root
 |-- cliente: string (nullable = true)
 |-- id_lote: integer (nullable = true)
 |-- id_conta: integer (nullable = true)
 |-- id_item: integer (nullable = true)
 |-- servico: long (nullable = true)
 |-- descricao_despesa: string (nullable = true)
 |-- valor: string (nullable = true)
 |-- qtde: double (nullable = true)



Corrigindo campo "valor":

In [6]:
df = df.withColumn("valor", df["valor"].cast("double"))

Selecionando os "id_conta" atrelados a pelo menos um serviço além do TUSS mestre:

In [7]:
service_counts = df.groupBy("id_conta").count()
ids = service_counts.filter(service_counts["count"] != 1).rdd.map(lambda (acc_id, count): acc_id).collect()

Montando uma lista com todos os serviços existentes no dataset:

In [8]:
services_set = set(df.select("servico").rdd.flatMap(lambda x: x).collect())

À partir dos IDs selecionados, montamos um dicionario que guarde a contagem de cada serviço para cada id_conta:

In [9]:
services_dict = {}
for acc_id in ids:
    services_dict[acc_id] = Counter(df.filter(df["id_conta"] == acc_id).select("servico").rdd.flatMap(lambda x: x).collect())

Definindo uma função que, dado um serviço, compute a porcentagem de id_conta's na qual esse serviço aparece e a media de aparições do serviço por id_conta. 

In [10]:
def compute_frequency(service):
    accs_occur = 0 # id_conta's em que o servico aparece
    total_occur = 0 # total de vezes queo servico aparece (pode ser mais de uma por id conta)
    for acc in services_dict.keys():
        if services_dict[acc][service] != 0:
            accs_occur += 1
            total_occur += services_dict[acc][service]
    return ((float(accs_occur) * 100) / len(ids), float(total_occur) / len(ids)) 

Gerando dicionario com as frequencias de todos os serviços

In [11]:
freq_dict = {}
for service in services_set:
    freq_dict[service] = compute_frequency(service)

Função que seleciona serviços que aparecem em pelo menos X% dos id_conta's:

In [12]:
def get_highest(x):
    '''
    Retorna lista de listas com codigo do servico, porcentagem de contas em que ele aparece, media de aparicoes por conta e
    descricao.
    '''
    line_list = []
    for service in freq_dict.keys():
        if freq_dict[service][0] > x: # freq_dict[service][0] : porcentagem, [1]: media
            line = [service, freq_dict[service][0], freq_dict[service][1] , df.filter(df["servico"] == service).select("descricao_despesa").rdd.flatMap(lambda x: x).take(1)[0]]
            line_list.append(line)
    return line_list

Selecionando os servicos que aparecem em mais de 30% dos id_conta e transformando em uma dataframe pandas para facilitar visualização:

In [13]:
greater_than_30 = sorted(get_highest(20), key=lambda x: x[1], reverse=True) # ordena por porcentagem, maior p/ menor

In [14]:
freq_df = DataFrame(greater_than_30, columns=["Codigo", "%", "Frequencia", "Descricao"])

In [15]:
freq_df

,Codigo,%,Frequencia,Descricao
0,30724058,100.000000,8.424242,ARTOPLASTIA FEMORAL COXO FEMORAL
1,40304361,42.424242,3.333333,HEMOGRAMA COM CONTAGEM DE PLAQUETAS OU FRACOES...
2,60023155,33.333333,0.818182,"TAXA DE SALA CIRURGICA, PORTE ANESTESICO 6"
3,90008332,33.333333,1.636364,KEFAZOL 1 G. FR/AMP
4,40301630,33.333333,3.000000,CREATININA - PESQUISA E/OU DOSAGEM
5,40302318,33.333333,3.151515,POTASSIO - PESQUISA E/OU DOSAGEM
6,40302423,33.333333,3.060606,SODIO - PESQUISA E/OU DOSAGEM
7,40302580,33.333333,2.909091,UREIA - PESQUISA E/OU DOSAGEM
8,10102019,30.303030,4.787879,VISITA HOSPITALAR (PACIENTE INTERNADO)
9,10104020,30.303030,2.575758,ATENDIMENTO MEDICO DO INTENSIVISTA EM UTI GERA...
